# Spooky Author Indentification
- Give the probability that an author has written a text

Load libraries

In [1]:
import pandas as pd
import numpy as np
import sklearn.datasets
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import SGDClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.cross_validation import KFold
from sklearn.metrics import confusion_matrix, f1_score
from sklearn.model_selection import GridSearchCV
from bs4 import BeautifulSoup
import re
import string
import nltk
from nltk.tokenize import WordPunctTokenizer
tok = WordPunctTokenizer()
from sklearn.metrics import confusion_matrix
#import ConfusionMatrix
from pandas_ml import ConfusionMatrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix

/Users/laurent/anaconda/lib/python3.6/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


Load Data

In [26]:
train = pd.read_csv("train.csv", encoding='ISO-8859-1')
test = pd.read_csv("test.csv", encoding='ISO-8859-1')

#### Function to clean Data

In [4]:
pat1 = r'@[A-Za-z0-9]+'
pat2 = r'https?://[A-Za-z0-9./]+'
combined_pat = r'|'.join((pat1, pat2))
def tweet_cleaner(text):
    soup = BeautifulSoup(text, 'lxml')
    souped = soup.get_text()
    stripped = re.sub(combined_pat, '', souped)
    try:
        clean = stripped.decode("utf-8-sig").replace(u"\ufffd", "?")
    except:
        clean = stripped
    letters_only = re.sub("[^a-zA-Z]", " ", clean)
    lower_case = letters_only.lower()
    # During the letters_only process two lines above, it has created unnecessay white spaces,
    # I will tokenize and join together to remove unneccessary white spaces
    words = tok.tokenize(lower_case)
    return (" ".join(words)).strip()

## Clean Train Dataset

In [13]:
training = train.text
dftext = []
for t in training:
    dftext.append(tweet_cleaner(t))

Create dataframe with train data

In [14]:
dfList = train['author'].tolist()

In [15]:
dataset = sklearn.datasets.base.Bunch(data=dftext, target=dfList)
train = pd.DataFrame(dataset)
train.columns = ['text','author']

## Create basic Pipeline

Multinomial Naive Bayesian

In [9]:
pipeline = Pipeline([
    ('count_vectorizer',   CountVectorizer()),
    ('tfidf_transformer',  TfidfTransformer()),
    ('classifier',         MultinomialNB(alpha = 0.01))
])

### Test prediction with train data

In [18]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(train.text, train.author, test_size=0.33, random_state=42)

In [33]:
model = pipeline.fit(X_train, y_train)
y_pred = model.predict_proba(X_test)

### Confusion Matrix

In [ ]:
cm = confusion_matrix(y_test, y_pred)

### Clean Test Data

In [34]:
testing = test.text
dftest = []
for t in testing:
    dftest.append(tweet_cleaner(t))

### Prediction on Test Data
- Probabilities

In [29]:
prediction = model.predict_proba(dftest)

In [31]:
result=pd.DataFrame()
result["id"]=test["id"]
result["EAP"]=prediction[:,0]
result["HPL"]=prediction[:,1]
result["MWS"]=prediction[:,2]
result

,id,EAP,HPL,MWS
0,id02310,0.061571,0.000587,0.937842
1,id24541,0.976762,0.016343,0.006895
2,id00134,0.323581,0.674942,0.001477
3,id27757,0.622550,0.376480,0.000969
4,id04081,0.857830,0.106108,0.036062
5,id27337,0.743494,0.215303,0.041204
6,id24265,0.593116,0.234158,0.172726
7,id25917,0.099830,0.048839,0.851331
8,id04951,0.996612,0.003349,0.000039
9,id14549,0.802137,0.182535,0.015328


### Download predictions

In [35]:
result.to_csv("sub1.csv", index=False)